## Mining argumentative content from news articles

When researching a topic, you often can search and retrieve thousands of articles that relate to the given topic.
Finding the most relevant segment in these articles can be time consuming.

This notebook demonstrates how to use Project Debater APIs to extract the most relevant argumentative statements for a given topic from a corpus of articles on the topic.  

As an example, it uses the Watson Discovery News (https://www.ibm.com/watson/services/discovery-news/) to retrieve news articles on the provided topic.  However, it is very simple to adapt it to import data from  other source such as other search engines or collection of articles that reside in text files.

The output includes three main parts:
1. A table of the most argumentative statements with respect to the topic
2. A narrative which automatically generated from the most argumentative statements
3. Given an additional list of aspects of interest it provides a summary of the most argumentative statements for each aspect seperately.


The following two lines define the topic of interest and its specific aspects that are of interest.  In this case, we are interested in pros and cons related to Hybrid Cloud in general, and also those that related to the price, value , investment in Hybrid Cloud as well as IBM and Microsoft specific statements.

In [1]:
dc = 'Hybrid cloud'
aspects = ['Price','Value','Investment','IBM','Microsoft']

These are additional parameters, which can remain as default in most cases:

In [2]:
articles_filename = dc + ".json"
dc_re = dc  # This can allow a more refined regular expression to find relevant sentences containing the topic concept.
            # For example 'Hybrid.*Cloud' would increase coverage.
topic = 'We should support ' + dc
max_sentence_length = 50
max_articles = 10000


### Query from Watson Discovery News

We query Watson Discovery News for articles containing the given topic and store the results in a file (to avoid reruning the query every time the notebook is executed).  

To use the Watson Discovery News service, you first need an IBM Cloud Account and then to provision a Watson Discovery SaaS instance.  You can sign up for free in  https://www.ibm.com/watson/services/discovery-news/.

Alternatively, you can replace Watson Discovery News with any other source of articles, just make sure the 'texts' variable contains a list of dictionaries with the the following keys for each article:

news_content - text of article
news_title - title of article
id - unique textual identifier of article

To run a Watson Discovery News query, two environment variables need to be set before running the notebook:

DISCOVERY_URL and DISCOVERY_APIKEY.  

These are found in the credentials sections of the Discovery Service in your IBM Cloud account.

Note that if you are using the LITE free Discovery plan, you are limited in the amounts of Watson Discovery news queries per month, and can not run the example more than once or twice.


In [3]:
import os
import requests
import urllib.parse
import json
def query_discovery_news(concept,offset):
    url =  os.environ['DISCOVERY_URL'] + '/v1/environments/system/collections/news-en/query?version=2019-04-30&count=50&offset='+str(offset)+'&query=text:'+urllib.parse.quote(concept)
    print(url)
    api_key = os.environ['DISCOVERY_APIKEY']
   
    resp = requests.get(url, verify=False, auth=('apikey', api_key))
    if resp.status_code != 200:
        print('GET to %s failed: %d: %s' % (url, resp.status_code, resp.reason))
        print(resp.text)
        return {}
    return resp.json()

offset = 0
texts = []
max_offset = max_articles
while (offset < max_offset):
    results = query_discovery_news(dc,offset)
    max_offset = min (max_articles,results['matching_results'])
    texts.extend ([ { 'news_content': result['text'], 'news_title' : result['title'], 'id' : result['id'] } for result in results['results'] ])
    print(len(texts))
    offset = offset + 50


https://api.us-south.discovery.watson.cloud.ibm.com/instances/5049511a-b775-4704-865b-8a971a76c862/v1/environments/system/collections/news-en/query?version=2019-04-30&count=50&offset=0&query=text:Hybrid%20cloud


/Users/katz/.local/lib/python3.7/site-packages/urllib3-1.24.3-py3.7.egg/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


50
https://api.us-south.discovery.watson.cloud.ibm.com/instances/5049511a-b775-4704-865b-8a971a76c862/v1/environments/system/collections/news-en/query?version=2019-04-30&count=50&offset=50&query=text:Hybrid%20cloud


/Users/katz/.local/lib/python3.7/site-packages/urllib3-1.24.3-py3.7.egg/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


100
https://api.us-south.discovery.watson.cloud.ibm.com/instances/5049511a-b775-4704-865b-8a971a76c862/v1/environments/system/collections/news-en/query?version=2019-04-30&count=50&offset=100&query=text:Hybrid%20cloud


...

9900
https://api.us-south.discovery.watson.cloud.ibm.com/instances/5049511a-b775-4704-865b-8a971a76c862/v1/environments/system/collections/news-en/query?version=2019-04-30&count=50&offset=9900&query=text:Hybrid%20cloud


/Users/katz/.local/lib/python3.7/site-packages/urllib3-1.24.3-py3.7.egg/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


9950
https://api.us-south.discovery.watson.cloud.ibm.com/instances/5049511a-b775-4704-865b-8a971a76c862/v1/environments/system/collections/news-en/query?version=2019-04-30&count=50&offset=9950&query=text:Hybrid%20cloud


/Users/katz/.local/lib/python3.7/site-packages/urllib3-1.24.3-py3.7.egg/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


10000


Save the results to file.

In [4]:
with open(articles_filename, 'w') as outfile:
    json.dump(texts, outfile)  
 

### Initialize the Debater API 

The next segment initializes the Debater API.  It assumes the DEBATER_API_KEY environment variable was set with the Project Debater Early Access Program API key, before the notebook was loaded.

In [5]:
# (C) Copyright IBM Corp. 2020.
import os
from debater_python_api.api.clients.narrative_generation_client import Polarity
from debater_python_api.api.debater_api import DebaterApi
api_key = os.environ['DEBATER_API_KEY']
debater_api = DebaterApi(api_key)

The following are some utility functions to pretty print results.

In [6]:
from IPython.core.display import HTML,display
import math
def print_candidates(title,candidates,fields = None):
    display(HTML('<h2>'+title+'</h2>'))
    print('Number of candidates: {}'.format(len(candidates)))
    html = "<table>"
    html += "<tr>"
    if fields == None:
        fields = candidates[0]
    for field in fields:
       html += "<td><p style=\"text-align:left;\">%s</p></td>"%(field)
    html += "</tr>"

    for row in candidates:
        html += "<tr>"
        for field in fields:
            html += "<td><p style=\"text-align:left;\">%s</p></td>"%(row[field])
        html += "</tr>"
    html += "</table>"
    display(HTML(html))     

def print_top_by_function(title,candidates,function, n = 20, fields = None):
    sorted_candidates = sorted(candidates, key=function)[0:n]
    print_candidates(title,sorted_candidates,fields)
    
    

### Read the articles from the saved file

Next we, read the articles from the saved file in json format, split into to sentences, and create an html snippet which includes the sentence, the surrounding context and the article's title.  We also remove duplicate sentences.

In [7]:
import json
with open(articles_filename, "r") as read_file:
    data = json.load(read_file)

from spacy.lang.en import English # updated
import re
nlp = English()

all_sentences = []
hashed_sentences = {}
total_sentences = 0
nlp.max_length = 1000000000
nlp.add_pipe(nlp.create_pipe('sentencizer')) # updated
for hit in data:
    doc_id = hit['id']
    doc_text = hit['news_content']
    doc_title = hit['news_title']
    doc = nlp(doc_text)
    prev = {'text' : '', 'html':''}
    for (i,sent) in enumerate(doc.sents):
        total_sentences += 1
        text = sent.string.strip()
        hash_of_text = hash(text)
        if hash_of_text in hashed_sentences:
            continue
        hashed_sentences[hash_of_text] = True    
        sentence_entry = { 'doc_id': doc_id , 'line' : i , 'text': text , 'prev': prev['text'] }
        sentence_entry['wordcount'] = len(re.findall(r'\w+', sentence_entry['text']))
        prev[ 'next' ] = sentence_entry['text']
        sentence_entry[ 'html'] = '<i>' + doc_title +'</i><br><br>' + sentence_entry['prev'] + '<b><br>' + sentence_entry['text'] + '</b><br>' 
        prev[ 'html' ] += sentence_entry['text']
        prev = sentence_entry
        all_sentences.append(sentence_entry)
    sentence_entry['next'] = ''      
print('Number of removed duplicates: {}'.format(total_sentences - len(all_sentences)))
print('Total number of sentences remaining: {}'.format(len(all_sentences)))



Number of removed duplicates: 6695
Total number of sentences remaining: 10207


We now filter sentences that contain the main concept using a regular expression. We also remove sentences that are too long.


In [8]:
import re
all_candidates = [sent for sent in all_sentences  if re.search(dc_re,sent['text'],re.IGNORECASE) and sent['wordcount'] < max_sentence_length]
print('Total number of sentences containing the topic:' + str(len(all_candidates)))
print_candidates("Example candidates", all_candidates[0:100], ['html'])

Total number of sentences containing the topic:2723


Number of candidates: 100


html
"5 Advantages of Using a Hybrid Cloud StrategyThis means that a hybrid cloud infrastructure can be a more compliant solution, depending on your business needs.Uniformity Hybrid clouds are becoming more important to public cloud providers."
"5 Advantages of Using a Hybrid Cloud StrategyThis means that a hybrid cloud infrastructure can be a more compliant solution, depending on your business needs.Uniformity Hybrid clouds are becoming more important to public cloud providers.Challenges faced with hybrid clouds are incorporating both the public and private clouds."
"5 Advantages of Using a Hybrid Cloud StrategyUniformity Hybrid clouds are becoming more important to public cloud providers.Challenges faced with hybrid clouds are incorporating both the public and private clouds.However, many public cloud providers are choosing to"
"What is hybrid cloud? Everything you need to knowHybrid cloud migration strategies to consider include lift and shift, refactoring and redesign.Hybrid cloud use cases Before you implement a hybrid cloud model, review common hybrid cloud use cases to determine whether this approach fits your company's IT needs."
"What is hybrid cloud? Everything you need to knowHybrid cloud migration strategies to consider include lift and shift, refactoring and redesign.Hybrid cloud use cases Before you implement a hybrid cloud model, review common hybrid cloud use cases to determine whether this approach fits your company's IT needs.Digital transformation."
Need for Hybrid Cloud Technology Services in 2021Perks of Hybrid Cloud Technology Hybrid cloud technology gives a business the flexibility to deploy its most sensitive and secure workloads in an on-premises cloud and to host less-critical resources on third-party public cloud providers.
CyrusOne’s Hybrid Cloud Business Strategy Leans On the BasicsCloud Computing > Colocation CyrusOne’s Hybrid Cloud Business Strategy Leans On the Basics In the next installment of our series exploring colocation providers’ strategies around hybrid cloud we look at CyrusOne.
Hybrid Cloud 102: Best Practices for Hybrid Cloud Enterprise ConfigurationEDT Hybrid Cloud 102: Best Practices for Hybrid Cloud Enterprise Configuration 11 a.m. PDT / 2 p.m. EDT Hybrid Cloud 103: Hottest Third-Party Tools Register now!
"Storage Infrastructure in a Hybrid Cloud WorldStorage > Cloud Storage Storage Infrastructure in a Hybrid Cloud World This Omdia report provides advice for enterprises looking to make best use of storage infrastructure in a hybrid cloud environment.Dennis Hahn | Jun 17, 2021 This is an extract from Omdia’s Data Center Storage in a Hybrid Cloud"
"Storage Infrastructure in a Hybrid Cloud WorldStorage > Cloud Storage Storage Infrastructure in a Hybrid Cloud World This Omdia report provides advice for enterprises looking to make best use of storage infrastructure in a hybrid cloud environment.Dennis Hahn | Jun 17, 2021 This is an extract from Omdia’s Data Center Storage in a Hybrid Cloud"


We now score the candidate sentences' stance toward the topic using the Debater Pro/Con service.  

In [9]:
sentence_topic_dicts = [{'sentence' : sentence['text'], 'topic' : topic } for sentence in all_candidates]
pro_con_scores = debater_api.get_pro_con_client().run(sentence_topic_dicts)
for sentence, pro_con_score in zip(all_candidates, pro_con_scores):
    sentence['procon'] = round(pro_con_score, 3)


ProConClient: 100%|██████████| 2723/2723 [00:10<00:00, 246.05it/s]

Similarly, we score the candidate sentences according to the Debater Evidence Detection service.

In [10]:
evidence_scores = debater_api.get_evidence_detection_client().run(sentence_topic_dicts)
for sentence, evidence_score in zip(all_candidates, evidence_scores):
    sentence['evidence'] = round(evidence_score, 3)


EvidenceDetectionClient: 100%|██████████| 2723/2723 [01:24<00:00, 28.52it/s] 

We now print the top 20 statements that have the highest combination of evidence and stance scores.

In [11]:
print_top_by_function("Most argumentative",all_candidates,
                          function=lambda candidate: -abs(candidate['procon'])-candidate['evidence'],
                                                       fields = ['html','procon','evidence'])

Number of candidates: 20


html,procon,evidence
"Overcoming Security as a Barrier to Cloud AdoptionA recent report found that nearly 94 percent of organizations responding to the survey agreed that the hybrid cloud is critical for meeting their immediate business needs.More than six in 10 of respondents said they are already using or piloting hybrid cloud services, with another third planning to roll",0.999,0.998
"Challenges Faced in South Africa Haven’t Stopped Its Race to Join a Digital Future – IT News AfricaAccording to the third annual Nutrix Enterprise Cloud Index, the vast majority of IT respondents in South Africa (88%) and globally (87%) reported that hybrid cloud is the ideal infrastructure model for their organisation.",0.999,0.998
"Challenges Faced in South Africa Haven't Stopped Its Race to Join a Digital FutureHowever, cost implications are a key factor in harmonising and upgrading existing IT infrastructure or migrating workloads to the cloud.According to the third annual Nutrix Enterprise Cloud Index, the vast majority of IT respondents in South Africa (88%) and globally (87%) reported that hybrid cloud is the ideal infrastructure model",0.999,0.998
"Challenges don’t stop SA in race to digital futureAccording to the third annual Nutrix Enterprise Cloud Index, the vast majority of IT respondents in South Africa (88%) and globally (87%) reported that a hybrid cloud is the ideal infrastructure model for their organisation.",0.999,0.997
"Asian cloud providers maintain strong reputation in the regionA vast majority (85%) of businesses feel that cloud-native solutions have helped them cope with the impact of the pandemic and 69% reported that a hybrid cloud approach supported them in disaster recovery and business continuity planning.This proved especially true in regions like Indonesia, where 64% of businesses see",0.999,0.996
"CloudBolt Enhances Cost Management, Compliance and Governance, and Codeless IT Integrations with New Technology ReleaseAccording to the CloudBolt Industry Insights (CII) report , The Truth About Hybrid Cloud and Digital Transformation , 94% of ITOps leaders said that hybrid cloud is critical to digital transformation.Specifically, the top three initiatives most critical for hybrid cloud include: accelerating automation initiatives, optimizing cloud spend, and enabling",0.998,0.994
"CloudBolt responds to demand for hybrid cloud and automationAccording to the CloudBolt Industry Insights (CII) report, The Truth About Hybrid Cloud and Digital Transformation, 94% of ITOps leaders said that hybrid cloud is critical to digital transformation.Specifically, the top three initiatives most critical for hybrid cloud include: accelerating automation initiatives, optimising cloud spend, and enabling self-service IT.",0.998,0.994
"IBM Tops Second-Quarter Earnings Forecast, Cloud Revenue ImpressesCEO Arvind Krishna says 'client adoption of our hybrid cloud platform contributed to strong performance in Global Business Services and software and drove improved overall revenue growth.'Author: Martin Baccardax International Business Machines ( IBM ) - Get Report posted stronger-than-expected second-quarter earnings Monday as cloud revenue and operating margins",0.999,0.992
"KKR Completes Acquisition of EnsonoThe 3 Biggest Hybrid Cloud Potholes on the Road to Digital Transformation June 03, 2021 IT leaders across the board agree that hybrid cloud is essential for digital transformation (94%), according to the The Truth About Hybrid Cloud and Digital Transformation , a report from CloudBolt and social research platform",0.996,0.993
VST ECS Tech Summit 2021: The Future of Business“Red Hat believes that the open hybrid cloud can help solve and provide business and enterprise problems: Flexibility + Consistency + Efficiency.Red Hat has a vision for open hybrid cloud for nearly seven years already.,0.999,0.989


### Generate narrative

We now use the narrative generation service to automatically generate a short supporting narrative for the topic, based on the most argumentative evidence found.

In [12]:
# customize opening statement
opening_statement_customization_dict = {
    'type' : 'openingStatement',
    'items':
        [
              {
                'key': 'Opening_statement',
                'value': ' The following speech is based on <NUM_SBC_ARGUMENTS_SUGGESTED> mined from Watson Discovery News supporting the notion that <MOTION>.',
                'itemType' : 'single_string'
              },        
        ]
}

print('Generating speech:')

sentences = [ sentence['text'] for sentence in all_candidates if sentence['evidence'] > 0.7]
print('Number of sentences passed to speech generation: ' + str(len(sentences)))
narrative_generation_client = debater_api.get_narrative_generation_client()
sentence_topic_dicts = [{'sentence' : sentence, 'topic' : topic } for sentence in sentences]
pro_con_scores = debater_api.get_pro_con_client().run(sentence_topic_dicts)
speech = narrative_generation_client.run(topic=topic, dc=dc, sentences=sentences,
                                         pro_con_scores=pro_con_scores, polarity=Polarity.PRO, 
                                         customizations=[opening_statement_customization_dict])
print('\n\nSpeech:' + speech.status)
if (speech.status == 'DONE'):
    print(speech)



ProConClient:   0%|          | 0/93 [00:00<?, ?it/s]

Generating speech:
Number of sentences passed to speech generation: 93



ProConClient: 100%|██████████| 93/93 [00:00<00:00, 120.07it/s]

21/07/2021 16:56:25, SpeechResultResponse=60f82788ed7ca41fdb889561, motionGenerationid=0, status=RUNNING, started to wait at 21/07/2021 16:56:24
21/07/2021 16:56:35, SpeechResultResponse=60f82788ed7ca41fdb889561, motionGenerationid=0, status=RUNNING, started to wait at 21/07/2021 16:56:24


ProConClient: 100%|██████████| 93/93 [00:24<00:00,  3.78it/s] 



Speech:DONE
The following speech is based on 93 mined from Watson Discovery News supporting the notion that We should support Hybrid cloud. 

Let's turn to four issues mentioned by the crowd: ecosystems, businesses, digital transformation and investments. 

Let's talk about ecosystems.  The IBM Cloud for Financial Services with a control framework to help financial institutions accelerate hybrid cloud adoption and drive revenue growth while addressing their need for a secure and compliant partner ecosystem," said Brendan Kinkade, Vice President, Technology and Hybrid Cloud Partnerships, IBM.  

The next issue is businesses.  The report finds that together with investments in public, private and hybrid cloud solutions, the Omani economy will enable businesses to generate nearly RO 387 million in net new revenues over the next five years.   A vast majority of businesses feel that cloud-native solutions have helped them cope with the impact of the pandemic and 69% reported that a hybrid

### Analyze statements per user provided aspects

The next step is to extract for each sentence all the wikipedia concepts mentioned in it using the TermWikifier service.

In [13]:
def add_sentence_mentions(sentences):
    term_wikifier_client = debater_api.get_term_wikifier_client()
    mentions_list = term_wikifier_client.run([sentence['text'] for sentence in sentences])
    for sentence, mentions in zip(sentences, mentions_list):
        sentence['mentions'] = set([mention['concept']['title'] for mention in mentions])
    all_mentions = set([mention for sentence in sentences for mention in sentence['mentions']])
    return set(all_mentions)

all_mentions = add_sentence_mentions(all_candidates)    
print('Total number of unique concepts: {}'.format(len(all_mentions)))


EvidenceDetectionClient: 100%|██████████| 2723/2723 [01:56<00:00, 23.30it/s]

TermWikifierClient: 100%|██████████| 2723/2723 [00:37<00:00, 72.94it/s]

Total number of unique concepts: 2105


For each aspect, we find all related wikipedia concepts (that are mentioned in the given sentences) .

In [14]:
def get_related_mentions(concept, threshold, mentions):
    term_relater_client = debater_api.get_term_relater_client()
    concept_mention_pairs = [[concept, mention] for mention in mentions]
    scores = term_relater_client.run(concept_mention_pairs)
    return [mention for mention, score in zip(all_mentions, scores) if score > threshold]


In [15]:
matched_mentions = {}
for aspect in aspects:
    matched_mentions[aspect] = get_related_mentions(aspect, 0.7, all_mentions)
    print(aspect,":",matched_mentions[aspect])
    
    

TermRelaterClient:   0%|          | 0/2105 [00:00<?, ?it/s]

Price : ['Quantity', 'Revenue', 'Cash', 'Retail', 'Pricing', 'Market value', 'Trade', 'Demand', 'Dollar', 'Value-based pricing', 'PricewaterhouseCoopers', 'Payment', 'Economics', 'Purchasing', 'Sales', 'Profit (economics)', 'Tax', 'Cost', 'Price', 'Expense']


TermRelaterClient:   0%|          | 0/2105 [00:00<?, ?it/s]

Value : ['Institute for Business Value', 'Quantity', 'Value-added service', 'Coin', 'Market value', 'Creating shared value', 'Value (ethics)', 'Value proposition', 'Value-based pricing', 'Payment', 'Economics', 'Financial ratio', 'Value chain', 'Customer value proposition', 'Utility', 'Cost', 'Tax assessment', 'Business value']



TermRelaterClient:   0%|          | 0/2105 [00:00<?, ?it/s]

Investment : ['Investor', 'Investment strategy', 'Investment', 'Funding', 'Finance', 'Financial market', 'Financial services', 'Asset', 'Expense', 'Financial institution']


TermRelaterClient:   0%|          | 0/2105 [00:00<?, ?it/s]

IBM : ['Oracle Corporation', 'IBM Global Services', 'IBM WebSphere', 'IBM PC compatible', 'IBM Z', 'Computing platform', 'Computer', 'IBM FlashSystem', 'IBM Fellow', 'IBM Cloud', 'Think (IBM)', 'Technology company', 'IBM Research', 'Computing', 'IBM', 'Cloud computing', 'IBM India', 'IBM Award']



TermRelaterClient: 100%|██████████| 2105/2105 [00:06<00:00, 337.95it/s]

Microsoft : ['Microsoft Access', 'Oracle Corporation', 'Office 365', 'Intel', 'Microsoft Partner Network', 'Microsoft Defender', 'Microsoft Exchange Server', 'Microsoft 365', 'Microsoft Windows', 'Microsoft Azure', 'Microsoft Visual Studio', 'Multinational corporation', 'Computer', 'Technology company', 'Computing', 'Microsoft Dynamics 365', 'Microsoft Certified Partner', 'Microsoft', 'Microsoft OneDrive', 'Build (developer conference)']


We identify for each sentence all related concepts for each aspect. 

In [16]:
for sentence in all_candidates:
    for aspect in aspects:
        sentence[aspect + '_concepts'] = sentence['mentions'].intersection(matched_mentions[aspect]) 
 

TermRelaterClient: 100%|██████████| 2105/2105 [00:06<00:00, 349.46it/s]



We now generate the final output.  We print the top 20 most relevant sentences for each aspect.

We first focus on sentences that contain atleast one related concept to the aspect. 

We then print the top sentences sorted by anaggregated score based on their evidence score, the pro con score, and the number of related concepts in the sentence.

In [17]:
for aspect in aspects:
    matched_sentences = [sentence for sentence in all_candidates if len(sentence[aspect + '_concepts']) >= 1]
           
    for sentence in all_candidates:
        sentence['score'] = len(sentence[aspect + '_concepts'])/3 + sentence['evidence'] + abs(sentence['procon'])
          
    print_top_by_function(aspect,matched_sentences,
                          function=lambda candidate: -candidate['score'],
                                                       fields = ['html','procon', 'evidence',aspect + '_concepts'])

Number of candidates: 20


html,procon,evidence,Price_concepts
"[Africa Cloud Review] Simon Ngunjiri: Cloud computing is shaping Africa’s technological infrastructure growthA new IBM study conducted by the International Data Corporation (IDC), also revealed that 84% of South African C-Suite are either pursuing or planning hybrid cloud strategies .According to the study, C-Suite executives in South Africa are prioritising the implementation of hybrid cloud strategies to benefit from flexibility, cost",0.995,0.971,{'Cost'}
"Platform as a service (PaaS) Global Market Report 2021 Featuring Major Players – Amazon Web Service, Salesforce Platform, IBM Cloud Foundry, Microsoft Azure and Google Cloud Function – ResearchAndMarkets.comCompanies in the industry are increasingly preferring hybrid cloud solutions to increase efficiency, innovation, and reduce costs.Hybrid Cloud refers to a cloud infrastructure environment that is a mixture of private cloud, on-premises computing and public cloud solutions.",0.996,0.965,{'Cost'}
"HPE expands GreenLake services in TaiwanHPE Taiwan chairman and managing director Jon Wang pointed out that enterprises have become keen on adopting hybrid cloud systems amid increasing demand for more flexible and instantaneous services.HPE Taiwan's revenues from GreenLake have been rising in the past few years, Wang said.",0.989,0.97,{'Demand'}
"Cloud services to create 15,800 jobs in Oman by 2024The report finds that together with investments in public, private and hybrid cloud solutions, the Omani economy will enable businesses to generate nearly RO 387 million in net new revenues over the next five years.IDC’s research further shows that the IT sector in Oman is expected to reach RO",0.985,0.946,{'Revenue'}
"Cloud Services to Create 15,800 Jobs in Oman by 2024It builds on more than a decade of analysis around the economic impact of IT on local economies.The report finds that together with investments in public, private and hybrid cloud solutions, the Omani economy will enable businesses to generate nearly RO 387 million (~US$1 billion) in net new revenues over",0.988,0.932,{'Revenue'}
"Avetra Boosts Digital Services for E&U Clients with Support from IBM Cloud""We are seeing the energy and utilities industry accelerating its cloud adoption, particularly to improve communication, function and accessibility to meet the demands of increasingly digitally-native consumers,"" said Evaristus Mainsah, General Manager, IBM Hybrid Cloud and Edge Ecosystem.",0.96,0.955,{'Demand'}
"Adoption of cloud services to create 15,800 jobs by 2024The report finds that together with investments in public, private and hybrid cloud solutions, the Omani economy will enable businesses to generate nearly RO387mn in net new revenues over the next five years.IDC’s research further shows that the IT sector in Oman is expected to reach RO406mn by 2024,",0.988,0.921,{'Revenue'}
"IBM Board Approves Increase in Quarterly Cash Dividend for the 26th Consecutive Year - Up News InfoArvind Krishna, IBM chairman and chief executive officer, said: “Client adoption of our hybrid cloud and AI technologies, strong cash generation and disciplined financial management all enable us to invest in the business while continuing to return capital to our shareholders.We remain committed to IBM’s secure and growing dividend.”",0.976,0.917,{'Cash'}
"Nutanix and ET Works brings the cloud to the Royal Opera HouseNutanix Sales Vice President & General Manager Dom Poloniecki said: “In addition to delivering efficiencies for the ROH, operating a hybrid cloud with Nutanix provides the additional benefit of a fully integrated system so, when it comes to upgrades, downtime is no longer needed; it is simply a case of",0.91,0.907,{'Sales'}
"International Business Machines : IBM REPORTS 2021 SECOND-QUARTER RESULTS (Form 8-K) | MarketScreenerIBM REPORTS 2021 SECOND-QUARTER RESULTS Improved Revenue Growth Led by Hybrid Cloud Platform Adoption, Consulting and Software Highlights Second Quarter: ● Revenue 

Number of candidates: 20


html,procon,evidence,Value_concepts
"IBM, IISc Bengaluru to work on hybrid cloud solutionsA 2020 study by the IBM Institute for Business Value on hybrid cloud said that the value derived from a hybrid or multi-cloud including on-premise cloud is 2.5 times more valuable compared to a single cloud, single platform approach.",0.983,0.858,"{'Institute for Business Value', 'Value (ethics)'}"
"[Africa Cloud Review] Simon Ngunjiri: Cloud computing is shaping Africa’s technological infrastructure growthA new IBM study conducted by the International Data Corporation (IDC), also revealed that 84% of South African C-Suite are either pursuing or planning hybrid cloud strategies .According to the study, C-Suite executives in South Africa are prioritising the implementation of hybrid cloud strategies to benefit from flexibility, cost",0.995,0.971,{'Cost'}
"Platform as a service (PaaS) Global Market Report 2021 Featuring Major Players – Amazon Web Service, Salesforce Platform, IBM Cloud Foundry, Microsoft Azure and Google Cloud Function – ResearchAndMarkets.comCompanies in the industry are increasingly preferring hybrid cloud solutions to increase efficiency, innovation, and reduce costs.Hybrid Cloud refers to a cloud infrastructure environment that is a mixture of private cloud, on-premises computing and public cloud solutions.",0.996,0.965,{'Cost'}
"Why hybrid cloud architecture is becoming a norm in the enterprise world - Express ComputerHybrid model is here to stay As per a recent survey by IBM Institute of Business Value (IBV), hybrid cloud adoption among Indian businesses is expected to grow by 49% with the average organisation using nearly six hybrid clouds.",0.991,0.866,{'Business value'}
"How IBM is building together across the tech ecosystem to enable developersAn IBM Institute for Business Value study, The hybrid cloud platform advantage found that a typical enterprise uses nearly 8 clouds from multiple vendors, and in the next 3 years, hybrid cloud adoption is expected to grow by 47%.The average organization will be using nearly 6 clouds.",0.989,0.722,{'Institute for Business Value'}
"IBM and Indian Institute of Science Launch Lab to Advance Hybrid Cloud Researchpower of hybrid cloud by producing faster, seamless and more secure adoption of hybrid cloud & artificial intelligence.Â A study from the IBM Institute for Business Value on hybrid cloud shows that the value from a hybrid multi-cloud platform technology and operating model at scale is 2.5 times the value",0.973,0.723,{'Institute for Business Value'}
"Top 3 Cloud Industry Trends for 2021 from Dell TechnologiesHowever, the need for consistent operations and infrastructure across clouds is paramount, so organizations are quickly finding that hybrid cloud models are the right strategy when it comes to longer-term costs, scalability and security.",0.987,0.699,{'Cost'}
"Innovation is a core value for Salesforce- Sunil Jose, Salesforce IndiaWhat sort of options do Salesforce customers have if they want to move to a virtualized cloud platform?Growing awareness of the efficiency in business processes through the use of hybrid cloud is exponentially increasing the amount of business data through multiple applications of the Internet of things across industries.",0.996,0.547,{'Quantity'}
"Avoid the mounting costs of multiplayer games with hybrid cloudWhile relying on the cloud might eat up your budget, relying on bare metal alone might not allow a studio to scale up fast enough if a game becomes popular.As the launch of a game is rarely predictable, using hybrid cloud offers the best way of keeping costs down.",0.999,0.478,{'Cost'}
"Five trends to watch out for in the realm of cloud computingHybrid cloud: One solvent for all cloud solutions By adopting a hybrid cloud, businesses can make a transition to the cloud at their own pace, at a lower cost and fewer risks.This solution combines multiple private and public clouds in the same system.",0.984,0.375,{'Cost'}


Number of candidates: 20


html,procon,evidence,Investment_concepts
"Hanzo Joins Growing Ecosystem of Partners Supporting the IBM Cloud for Financial Services""We designed the IBM Cloud for Financial Services with a control framework to help financial institutions accelerate hybrid cloud adoption and drive revenue growth while addressing their need for a secure and compliant partner ecosystem,"" said Brendan Kinkade, Vice President, Technology and Hybrid Cloud Partnerships, IBM.",0.96,0.797,"{'Financial institution', 'Financial services'}"
"Hanzo Joins Growing Ecosystem of Partners Supporting the IBM Cloud for Financial Services“We designed the IBM Cloud for Financial Services with a control framework to help financial institutions accelerate hybrid cloud adoption and drive revenue growth while addressing their need for a secure and compliant partner ecosystem,” said Brendan Kinkade, Vice President, Technology and Hybrid Cloud Partnerships, IBM.",0.95,0.787,"{'Financial institution', 'Financial services'}"
"Hanzo Joins Growing Ecosystem of Partners Supporting the IBM Cloud for Financial Servicesthe IBM Cloud for Financial Services with a control framework to help financial institutions accelerate hybrid cloud adoption and drive revenue growth while addressing their need for a secure and compliant partner ecosystem,” said Brendan Kinkade, Vice President, Technology and Hybrid Cloud Partnerships, IBM.",0.972,0.756,"{'Financial institution', 'Financial services'}"
"NSEIT buys majority stake in integrated cloud service provider Cloudxchange.io By CIOReviewIndia TeamAs per report, 63 per cent of India's enterprises have increased investments in hybrid cloud which is higher than the global average of 46 per cent.NSEIT is a global technology enterprise, focused on delivering excellence in a complex digital environment primarily in banking, insurance, capital market ecosystem and online",0.967,0.968,{'Investment'}
"Cloud services to create 15,800 jobs in Oman by 2024The report finds that together with investments in public, private and hybrid cloud solutions, the Omani economy will enable businesses to generate nearly RO 387 million in net new revenues over the next five years.IDC’s research further shows that the IT sector in Oman is expected to reach RO",0.985,0.946,{'Investment'}
"NSEIT acquires a majority stake in cloudxchange.ioAs per Nasscom’s report, 63% of India’s enterprises have increased investments in hybrid cloud which is higher than the global average of 46%.With this addition of Cloudxchange.io, NSEIT will offer end to end digital transformation solutions from Cloud to Security.",0.983,0.945,{'Investment'}
"Cloud Services to Create 15,800 Jobs in Oman by 2024It builds on more than a decade of analysis around the economic impact of IT on local economies.The report finds that together with investments in public, private and hybrid cloud solutions, the Omani economy will enable businesses to generate nearly RO 387 million (~US$1 billion) in net new revenues over",0.988,0.932,{'Investment'}
"NSEIT Acquires Majority Stake In Cloudxchange.ioAs per Nasscom's report, 63 per cent of India's enterprises have increased investments in hybrid cloud which is higher than the global average of 46 per cent.NSEIT is a global technology enterprise, focused on delivering excellence in a complex digital environment primarily in banking, insurance, capital market ecosystem and",0.963,0.955,{'Investment'}
"Get the latest in telecom and enterprise news and updates on Communications Today.In recent times, the entire industry has witnessed an exponential surge in cloud adoption especially during the pandemic.As per Nasscom’s report, 63 per cent of India’s enterprises have increased investments in hybrid cloud which is higher than the global average of 46 per cent.",0.97,0.943,{'Investment'}
"NSEIT buys majority stake in Cloudxchange.ioThe addition of new age cloud offerings of Cloudxchange.io to NSEIT’s digital transformation portfolio will further enhance its offerings t

Number of candidates: 20


html,procon,evidence,IBM_concepts
"IBM Tops Second-Quarter Earnings Forecast, Cloud Revenue ImpressesCEO Arvind Krishna says 'client adoption of our hybrid cloud platform contributed to strong performance in Global Business Services and software and drove improved overall revenue growth.'Author: Martin Baccardax International Business Machines ( IBM ) - Get Report posted stronger-than-expected second-quarter earnings Monday as cloud revenue and operating margins",0.999,0.992,"{'IBM Global Services', 'Cloud computing'}"
"IBM revenue jumps thanks to cloud growthIn the release, IBM chairman and CEO Arvind Krishna said client adoption of IBM’s hybrid cloud platform contributed to a strong performance in global business services and software during the second quarter.",0.991,0.966,"{'Cloud computing', 'IBM'}"
"IBM signs 10-year partnership with BIAL for digital transformation“IBM Global Business Services and Kyndryl will apply our expertise in hybrid cloud and building business platforms to help BIAL innovate, improve its operational efficiency and deliver exceptional experiences to its growing passenger base,"" said Mark Foster, senior vice president, IBM Services and IBM Global Business Services.",0.925,0.898,"{'IBM Global Services', 'IBM'}"
"International Business Machines : IBM REPORTS 2021 SECOND-QUARTER RESULTS (Form 8-K) | MarketScreenerIBM REPORTS 2021 SECOND-QUARTER RESULTS Improved Revenue Growth Led by Hybrid Cloud Platform Adoption, Consulting and Software Highlights Second Quarter: ● Revenue of $18.7 billion, up 3 percent (flat adjusting for divested businesses and currency)",0.951,0.84,"{'Cloud computing', 'IBM'}"
"Hanzo Joins Growing Ecosystem of Partners Supporting the IBM Cloud for Financial Services""We designed the IBM Cloud for Financial Services with a control framework to help financial institutions accelerate hybrid cloud adoption and drive revenue growth while addressing their need for a secure and compliant partner ecosystem,"" said Brendan Kinkade, Vice President, Technology and Hybrid Cloud Partnerships, IBM.",0.96,0.797,"{'IBM', 'IBM Cloud'}"
"Hanzo Joins Growing Ecosystem of Partners Supporting the IBM Cloud for Financial Services“We designed the IBM Cloud for Financial Services with a control framework to help financial institutions accelerate hybrid cloud adoption and drive revenue growth while addressing their need for a secure and compliant partner ecosystem,” said Brendan Kinkade, Vice President, Technology and Hybrid Cloud Partnerships, IBM.",0.95,0.787,"{'IBM', 'IBM Cloud'}"
"Hanzo Joins Growing Ecosystem of Partners Supporting the IBM Cloud for Financial Servicesthe IBM Cloud for Financial Services with a control framework to help financial institutions accelerate hybrid cloud adoption and drive revenue growth while addressing their need for a secure and compliant partner ecosystem,” said Brendan Kinkade, Vice President, Technology and Hybrid Cloud Partnerships, IBM.",0.972,0.756,"{'IBM', 'IBM Cloud'}"
"Second-quarter revenues beat forecasts at IBM - Sharecast.comArvind Krishna, chief executive, said: “Client adoption of our hybrid cloud platform contributed to the strong performance in global business services and software, and drove improved overall revenue growth. “We are pleased with our progress and we remain on track to deliver full-year revenue growth and meet out cash flow",0.998,0.987,{'Cloud computing'}
"Copersucar selects IBM cloud for sustainable digital growth in hybrid cloud environmentCopersucar selects IBM cloud for sustainable digital growth in hybrid cloud environment Anasia D’mello Copersucar, a global provider in sugar and ethanol trading has chosen IBM Cloud to help them increase productivity and efficiency, modernise processes and accelerate its business expansion.",0.982,0.661,"{'IBM', 'IBM Cloud'}"
"IBM and IISc launch research lab to advance hybrid cloud, AI technology adoptionAn in-house study by IBM on hybrid cloud reveals the hybrid value derived from multi-cloud plat

Number of candidates: 20


html,procon,evidence,Microsoft_concepts
Microsoft Azure Data Box available in Brazil for fast and cost-effective data transfer - Microsoft News Center Brazil - Mediarun SearchThe availability of Azure Data Box is linked to the efforts of Microsoft Mais Brasil’s plan to deliver hybrid cloud infrastructure and services to businesses to contribute to their growth and accelerate digital innovation by expanding cloud infrastructure in the country.,0.976,0.752,"{'Microsoft', 'Microsoft Azure'}"
Infrastructure Engineer JobInhouse development Salary - £50 000 - £60 000 Flexible remote working opportunities (2 days in the office a week) Role & Responsibilities Support and development of hybrid cloud infrastructure 3rd party support Embedding security in all work undertaken Skills & Qualifications Office 365 and Azure AD MS Windows Server,0.862,0.025,"{'Microsoft Windows', 'Office 365', 'Microsoft Azure'}"
"Function-as-a-Service Market to Reach $24.0 Billion by 2026On the other hand, the hybrid cloud segment is estimated to portray the highest CAGR of 34.8% during the forecast period.This is attributed to the increase in adoption of hybrid cloud in IT service management models and growing demand for cost-effective, scalable and agile computing.",0.967,0.34,{'Computing'}
"Microsoft aims to increase the security of its business with the acquisition of RiskIQIt will be possible to respond to. “Sign of the times “Microsoft provides end-to-end cloud-native security with Microsoft 365 Defender, Microsoft Azure Defender, and Microsoft Azure Sentinel to help protect, detect, and respond to threats in multi-cloud and hybrid cloud environments.",0.587,0.046,"{'Microsoft', 'Microsoft Azure', 'Microsoft 365'}"
"Microsoft : to acquire RiskIQ to strengthen cybersecurity of digital transformation and hybrid work | MarketScreenerMicrosoft has long been a leader in delivering end-to-end cloud-native security with Microsoft 365 Defender , Microsoft Azure Defender , and Microsoft Azure Sentinel that help protect, detect, and respond to threats in multi-cloud and hybrid cloud environments.",0.552,0.054,"{'Microsoft', 'Microsoft Azure', 'Microsoft 365'}"
"Get the latest in telecom and enterprise news and updates on Communications Today.Microsoft said now over 1,000 organizations are using Azure Arc primarily as a way to simplify the management of hybrid clouds, as well as run agile services across on-premises, multi-cloud and edge environments.",0.737,0.195,"{'Microsoft', 'Microsoft Azure'}"
"3 reasons CEOs should take their workforce to the cloudWith the big players already adapting to a post-pandemic future and 90% of global companies expecting to adopt a hybrid cloud by 2022, implementing WFH will be easy for enterprises.",0.948,0.313,{'Multinational corporation'}
"Nalgonda - Microsoft azure cloud partner in india | Azure cloud migration solutionsPublic Cloud Microsoft Azure Public Cloud built on Windows Server technology Private Cloud Virtual Private Cloud has physically isolated clusters with N+1 design.Hybrid Cloud Microsoft Azure offers scalability, flexibility and ease of use.https://alliancepro.co.in/microsoft/",0.993,0.264,{'Microsoft Azure'}
"AWS v Microsoft v Google: partners rank the Big 3The many points of entry into Azure for Microsoft customers “does separate Azure from the other cloud providers,” he said.Supporting hybrid cloud infrastructure is another strength for Microsoft, which embraced hybrid cloud earlier than AWS, Microsoft-focused solution providers said. “Microsoft created the gold standard in terms of operating in",0.994,0.222,{'Microsoft'}
"Ubuntu Blog: Finserv open source infrastructure powers digital transformationA hybrid cloud provides orchestration, management, and application portability between public and private clouds to create a single, flexible, optimal cloud infrastructure for running a financial institution’s computing workloads.",0.963,0.249,{'Computing'}
